[Unsloth](https://github.com/unslothai/unsloth)是Unsloth AI出品的一个高效的微调工具。它包含闭源和开源两个版本，在这里我们仅关注开源版本的使用。Unsloth重写了模型的内核，尤其是使用triton重写了loss、norm等算子，并手动重写了反向传播机制，使得模型的训练速度更快。同时，Unsloth使用了低精度量化结合LoRA进行微调，进一步降低了显存占用。

* Unsloth支持Llama、Mistral、Phi-3、Gemma、Yi、DeepSeek、Qwen、TinyLlama、Vicuna、Open Hermes等
* Unsloth支持16bit LoRA或4bit QLoRA。两者都快2倍。
* `max_seq_length`可以设置为任何值，因为通过了[kaiokendev的方法](https://kaiokendev.github.io/til)进行自动RoPE缩放。
* Unsloth使Gemma-2 9b / 27b **快2倍** 速度运行
* 支持自动导出到Ollama

要在您自己的计算机上安装Unsloth，请按照Github页面上的安装说明进行操作[这里](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions)。

笔记本中的功能：
1. 使用[FineTome 100K](https://www.modelscope.cn/datasets/AI-ModelScope/FineTome-100k)数据集进行训练。
1. 通过`standardize_sharegpt`将ShareGPT转换为标准格式
2. 通过`train_on_responses_only`仅在完成/助手上进行训练

In [1]:
# !pip install unsloth
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
# 截止到当前文件编写的时候，modelscope的pr目前没有合并进主分支，请使用这个命令安装unsloth：
!pip install git+https://github.com/tastelikefeet/unsloth.git@feat/modelscope

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
  Cloning https://github.com/tastelikefeet/unsloth.git (to revision feat/modelscope) to /tmp/pip-req-build-ylwo66va
  Running command git clone --filter=blob:none --quiet https://github.com/tastelikefeet/unsloth.git /tmp/pip-req-build-ylwo66va
  Running command git checkout -b feat/modelscope --track origin/feat/modelscope
  切换到一个新分支 'feat/modelscope'
  分支 'feat/modelscope' 设置为跟踪来自 'origin' 的远程分支 'feat/modelscope'。
  Resolved https://github.com/tastelikefeet/unsloth.git to commit 7146bc2f759a9a2f300f4009b1d3ab671bb8dbb5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


下面我们拉起模型。注意unsloth项目组在modelscope社区上提供了许多模型，如果您需要使用的模型在魔搭上不存在，请考虑告诉我们，或者直接上传一个。

In [1]:
import os
# 对于国内用户，魔搭社区提供了快速下载模型和数据集的方法，只需要简单引入一个环境变量:
os.environ['UNSLOTH_USE_MODELSCOPE'] = 'true'
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!
[2024-12-30 17:01:45,179] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
use modelscope


2024-12-30 17:01:46,115 - modelscope - INFO - Target directory already exists, skipping creation.


==((====))==  Unsloth 2024.12.9: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.347 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


现在添加LoRA适配器

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # 选择任何大于0的数字！建议8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 支持任何值，但=0是优化的
    bias = "none",    # 支持任何值，但="none"是优化的
    use_gradient_checkpointing = "unsloth", # 对于非常长的上下文使用True或"unsloth"
    random_state = 3407,
    use_rslora = False,  # 我们支持秩稳定的LoRA
    loftq_config = None, # 以及LoftQ
)

Unsloth 2024.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### 数据准备
我们现在使用 `Llama-3.1` 格式进行对话风格的微调。我们使用 [FineTome-100k](https://www.modelscope.cn/datasets/AI-ModelScope/FineTome-100k) 数据集，采用 ShareGPT 风格。但是我们将其转换为多轮格式 `("role", "content")`，而不是 `("from", "value")`。Llama-3 以如下方式呈现多轮对话：

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

我们使用 `get_chat_template` 函数来获取正确的聊天模板。我们支持 `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` 等。

In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from modelscope import MsDataset
dataset = MsDataset.load("AI-ModelScope/FineTome-100k", split = "train")

2024-12-30 17:03:36,084 - modelscope - INFO - storing https://www.modelscope.cn/api/v1/datasets/AI-ModelScope/FineTome-100k/repo?Source=SDK&Revision=master&FilePath=README.md&View=False in cache at /mnt/workspace/.cache/modelscope/hub/datasets/ad10352f0b6842676e6d5943f385c7bb2b07c1abd592c2aa260e9af638ecea41
2024-12-30 17:03:36,091 - modelscope - INFO - creating metadata file for /mnt/workspace/.cache/modelscope/hub/datasets/ad10352f0b6842676e6d5943f385c7bb2b07c1abd592c2aa260e9af638ecea41
2024-12-30 17:03:48,427 - modelscope - INFO - storing https://www.modelscope.cn/api/v1/datasets/AI-ModelScope/FineTome-100k/repo?Source=SDK&Revision=master&FilePath=data%2Ftrain-00000-of-00001.parquet in cache at /mnt/workspace/.cache/modelscope/hub/datasets/downloads/27484246eca7151cf2513ef6f4a6be4a7e7520bbcdd1b99b2cfd61c4164a7cd6
2024-12-30 17:03:48,436 - modelscope - INFO - creating metadata file for /mnt/workspace/.cache/modelscope/hub/datasets/downloads/27484246eca7151cf2513ef6f4a6be4a7e7520bbcdd1

现在使用 `standardize_sharegpt` 将 ShareGPT 风格的数据集转换为Conversations通用格式。这将数据集从如下格式:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
转换为
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [5]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 100000/100000 [00:09<00:00, 10242.12 examples/s]


我们来看一下第0项的对话结构：

In [6]:
dataset[0]["conversations"]

[{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
  'role': 'user'},
 {'content': 'Boolean operators a

<a name="Train"></a>
### 训练模型
作为例子，我们使用 Huggingface TRL 的 `SFTTrainer`进行训练：

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # 对于短序列可以使训练速度提高5倍。
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # 设置为1次完整的训练运行。
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # 用于WandB等
    ),
)

Map (num_proc=2): 100%|██████████| 100000/100000 [01:04<00:00, 1551.82 examples/s]


使用Unsloth的`train_on_completions`方法，仅对助手的输出进行训练，并忽略用户输入的损失。

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map: 100%|██████████| 100000/100000 [00:33<00:00, 2965.92 examples/s]


验证掩码是否实际完成：

In [15]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code t

In [16]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])
print(trainer.train_dataset[0]["labels"])

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

我们可以看到系统和指令提示已成功屏蔽。

In [17]:
#@title 显示当前内存状态
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. 最大内存 = {max_memory} GB.")
print(f"{start_gpu_memory} GB 的内存已保留。")

GPU = NVIDIA A100-SXM4-80GB. 最大内存 = 79.347 GB.
2.635 GB 的内存已保留。


In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.793500
2,0.839500
3,1.091000
4,0.904500
5,0.779700
6,0.924900
7,0.624100
8,1.003900
9,0.856700
10,0.752200


In [19]:
#@title 显示最终内存和时间统计
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} 秒用于训练。")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} 分钟用于训练。")
print(f"峰值保留内存 = {used_memory} GB。")
print(f"训练的峰值保留内存 = {used_memory_for_lora} GB。")
print(f"峰值保留内存占最大内存的百分比 = {used_percentage} %。")
print(f"训练的峰值保留内存占最大内存的百分比 = {lora_percentage} %。")

96.2098 秒用于训练。
1.6 分钟用于训练。
峰值保留内存 = 3.66 GB。
训练的峰值保留内存 = 1.025 GB。
峰值保留内存占最大内存的百分比 = 4.613 %。
训练的峰值保留内存占最大内存的百分比 = 1.292 %。


<a name="Inference"></a>
### 推理

In [20]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # 启用原生2倍速推理

messages = [
    {"role": "user", "content": "继续斐波那契数列：1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # 必须添加以进行生成
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n继续斐波那契数列：1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nContinue the Fibonacci sequence:\n\n9, 13, 21, 34, 55, 89<|eot_id|>']

 可以使用 `TextStreamer` 进行流式推理

In [21]:
FastLanguageModel.for_inference(model) # 启用本地2倍速度推理

messages = [
    {"role": "user", "content": "继续斐波那契数列：1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # 必须添加以进行生成
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

继续斐波那契数列：13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817


<a name="Save"></a>
### 保存和加载微调模型
要将最终模型保存为LoRA适配器，可以使用Huggingface的`push_to_hub`进行在线保存，或使用`save_pretrained`进行本地保存。

**[注意]** 这只保存LoRA适配器，而不是完整模型。要保存为16位或GGUF格式，请向下滚动！

In [22]:
model.save_pretrained("lora_model") # 本地保存
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # 在线保存
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # 在线保存

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

现在，如果你想加载我们刚刚保存的LoRA适配器进行推理，请将`False`设置为`True`：

In [23]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # 你用于训练的模型
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # 启用本地2倍速推理

messages = [
    {"role": "user", "content": "描述法国首都的一座高塔。"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # 必须添加用于生成
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

答案：埃夫兰塔高塔。<|eot_id|>


### 保存为 VLLM 的 float16

也支持直接保存为 `float16`。选择 `merged_16bit` 以保存为 float16 或选择 `merged_4bit` 以保存为 int4。我们还允许使用 `lora` 适配器作为后备选项

In [25]:
# 合并为16位
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

# 合并为4位
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)

# 仅LoRA适配器
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 142.73 out of 200.0 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 116.78it/s]


Unsloth: Saving tokenizer... Done.
Done.


### GGUF / llama.cpp 转换
Unsloth支持原生保存到 `GGUF` / `llama.cpp`。使用 `save_pretrained_gguf` 进行本地保存。

支持的部分量化方法如下：
* `q8_0` - 快速转换。高资源使用，但通常可以接受。
* `q4_k_m` - 推荐。对一半的 attention.wv 和 feed_forward.w2 张量使用 Q6_K，其他使用 Q4_K。
* `q5_k_m` - 推荐。对一半的 attention.wv 和 feed_forward.w2 张量使用 Q6_K，其他使用 Q5_K。

In [ ]:
# 保存为8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)

# 保存为16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")

# 保存为q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")